In [1]:
import numpy as np
import pandas as pd

In [386]:
import sys

In [394]:
with open('./task2_lemmas_train') as f:
    train = f.read().splitlines()

In [395]:
new_vals = []
for elem in train[1:]:
    vals = elem.split(',')[1:]
    for new_el in vals[1:]:
        new_el = new_el.split('+')
        new_vals.append((vals[0], new_el[0], new_el[1]))

In [396]:
train = pd.DataFrame(new_vals)
test = pd.read_csv('./task2_lemmas_test')

In [397]:
test.head()

,Id,X
0,1,gettonan
1,2,incidentali
2,3,involtino
3,4,lievi
4,5,comunistizzasse


In [398]:
train[0] = train[0].apply(lambda elem: elem.decode('utf-8'))
test['X'] = test['X'].apply(lambda elem: elem.decode('utf-8'))

In [399]:
123 

123

In [400]:
diff_lets = set()
for word in train[0]:
     for sym in word:
            diff_lets.add(sym)

In [401]:
train_clf = train[[0,2]]


,0,2
0,vergognerete,V
1,amnistiavate,V
2,menomazione,N
3,sfaldavamo,V
4,sfodererei,V
5,ascondesti,V
6,edifichereste,V
7,maschieran,V
8,transennasser,V
9,computando,V


In [403]:
vowels = {'a','e','y','u','i','o', 'A','E','Y','O','U','I', u'\xe0',
 u'\xe8',
 u'\xe9',
 u'\xec',
 u'\xf2',
 u'\xf9'} # мб стоит ё на е заменить везде
consonal = {'q','w','r','t','p','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m'}
consonal |= set(map(lambda let: let.upper(), consonal))
capsed = set(map(lambda let: let.upper(), consonal)) | set(map(lambda let: let.upper(), vowels))

In [405]:
train_clf['vowels'] = train_clf[0].apply(lambda elem: len(filter(lambda s: s in vowels, elem)))
train_clf['consonal'] = train_clf[0].apply(lambda elem: len(filter(lambda s: s in consonal, elem)))

# train_clf.drop('capsed', axis=1, inplace=True)

# #-----------#
test['vowels'] = test['X'].apply(lambda elem: len(filter(lambda s: s in vowels, elem)))
test['consonal'] = test['X'].apply(lambda elem: len(filter(lambda s: s in consonal, elem)))


In [406]:
from tqdm import tqdm

In [407]:
train_clf['length'] = map(len , train_clf[0])
test['length'] = map(len , test['X'])

doblets = []
for surn in tqdm(train_clf[0]):
    dobles = 0
    surn = surn.lower()
    for ind in range(1, len(surn)):
        if surn[ind] == surn[ind - 1]:
            dobles += 1
    doblets.append(dobles)
    
train_clf['doubles'] = doblets 

doblets = []
for surn in tqdm(test['X']):
    dobles = 0
    surn = surn.lower()
    for ind in range(1, len(surn)):
        if surn[ind] == surn[ind - 1]:
            dobles += 1
    doblets.append(dobles)
    
test['doubles'] = doblets 

100%|██████████| 29661/29661 [00:00<00:00, 160514.51it/s]


In [408]:
import scipy.sparse
from sklearn.preprocessing import PolynomialFeatures

In [409]:
sprsed_train = scipy.sparse.csr_matrix(train_clf.drop([0,2],axis=1).values)
sprsed_test = scipy.sparse.csr_matrix(test.drop(['Id', 'X'],axis=1).values)


In [410]:
from sklearn.feature_extraction.text import CountVectorizer

In [411]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 6), min_df=1) 
ngram_vectorizer.fit(train[0]) 

CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 6), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [412]:
ngram_train = ngram_vectorizer.transform(train[0])
ngram_test = ngram_vectorizer.transform(test['X'])


In [413]:
# ngram_train = ngram_vectorizer.transform(train[0])



all_sparsed_tr = scipy.sparse.bmat([[ngram_train, sprsed_train]], format='csr')
all_sparsed_te = scipy.sparse.bmat([[ngram_test, sprsed_test]], format='csr')


all_sparsed_tr.shape, all_sparsed_te.shape


((120897, 166200), (29661, 166200))

In [415]:
from sklearn.linear_model import LogisticRegression

In [418]:
train.head()

,0,1,2
0,vergognerete,vergognare,V
1,amnistiavate,amnistiare,V
2,menomazione,menomazione,N
3,sfaldavamo,sfaldare,V
4,sfodererei,sfoderare,V


In [419]:
algo_1 = LogisticRegression(C=.65).fit(all_sparsed_tr, train[2])
pos_predicted = algo_1.predict(all_sparsed_te)

In [421]:
pd.Series(pos_predicted).value_counts()

V    24379
A     3183
N     2099
dtype: int64

In [27]:
# ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=1) 
# ngram_train = ngram_vectorizer.fit_transform(train.surname) 

from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn import linear_model 


X_train, X_test, Y_train, Y_test = train_test_split(all_sparsed, train_clf[2], train_size=.8, random_state=42, stratify=train_clf[2]) 
lr = LogisticRegression(C=.65).fit(X_train, Y_train) 
print (accuracy_score(Y_test, lr.predict(X_test)))

0.946275693784


In [422]:
train_lem = train_clf.drop(2, axis=1)

In [423]:
train_lem

,0,vowels,consonal,length,doubles
0,vergognerete,5,7,12,0
1,amnistiavate,6,6,12,0
2,menomazione,6,5,11,0
3,sfaldavamo,4,6,10,0
4,sfodererei,5,5,10,0
5,ascondesti,4,6,10,0
6,edifichereste,6,7,13,0
7,maschieran,4,6,10,0
8,transennasser,4,9,13,2
9,computando,4,6,10,0


In [424]:
train_lem[1] = train[1]

In [425]:
train_lem[2] = train[2]

In [426]:
train_lem

,0,vowels,consonal,length,doubles,1,2
0,vergognerete,5,7,12,0,vergognare,V
1,amnistiavate,6,6,12,0,amnistiare,V
2,menomazione,6,5,11,0,menomazione,N
3,sfaldavamo,4,6,10,0,sfaldare,V
4,sfodererei,5,5,10,0,sfoderare,V
5,ascondesti,4,6,10,0,ascondere,V
6,edifichereste,6,7,13,0,edificare,V
7,maschieran,4,6,10,0,maschiare,V
8,transennasser,4,9,13,2,transennare,V
9,computando,4,6,10,0,computare,V


In [427]:
v_tra_lem = train_lem[train_lem[2] == 'V']
a_tra_lem = train_lem[train_lem[2] == 'A']
n_tra_lem = train_lem[train_lem[2] == 'N']

In [428]:
v_tra_lem.drop(2,axis=1, inplace=True)
a_tra_lem.drop(2,axis=1, inplace=True)
n_tra_lem.drop(2,axis=1, inplace=True)

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [429]:
def get_lcp(fst, snd):
    cur_len = 0
    while((cur_len < min(len(fst), len(snd)) and (fst[cur_len] == snd[cur_len]))):
        cur_len += 1
    
    return cur_len

In [430]:
get_lcp('asdwq', 'asd')

3

In [436]:
test.head()

,Id,X,vowels,consonal,length,doubles
0,1,gettonan,3,5,8,1
1,2,incidentali,5,6,11,0
2,3,involtino,4,5,9,0
3,4,lievi,3,2,5,0
4,5,comunistizzasse,6,9,15,2


In [437]:
v_tra_lem.head()

,0,vowels,consonal,length,doubles,1,resid,pref,lenka
0,vergognerete,5,7,12,0,vergognare,are,vergogn,5
1,amnistiavate,6,6,12,0,amnistiare,re,amnistia,4
3,sfaldavamo,4,6,10,0,sfaldare,re,sfalda,4
4,sfodererei,5,5,10,0,sfoderare,are,sfoder,4
5,ascondesti,4,6,10,0,ascondere,re,asconde,3


In [431]:
to_add = []
to_add_ = []
to_pref = []
for word, stemmed in v_tra_lem[[0,1]].values:
    lenka = get_lcp(word, stemmed)
    to_add.append(stemmed[lenka:])
    to_pref.append(word[:lenka])
    to_add_.append(len(word) - lenka)
v_tra_lem['resid'] = to_add
v_tra_lem['pref'] = to_pref
v_tra_lem['lenka'] = to_add_

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

In [432]:
to_add = []
to_add_ = []
to_pref = []
for word, stemmed in a_tra_lem[[0,1]].values:
    lenka = get_lcp(word, stemmed)
    to_add.append(stemmed[lenka:])
    to_pref.append(word[:lenka])
    to_add_.append(len(word) - lenka)
a_tra_lem['resid'] = to_add
a_tra_lem['pref'] = to_pref
a_tra_lem['lenka'] = to_add_

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipyk

In [433]:
to_add = []
to_add_ = []
to_pref = []
for word, stemmed in n_tra_lem[[0,1]].values:
    lenka = get_lcp(word, stemmed)
    to_add.append(stemmed[lenka:])
    to_pref.append(word[:lenka])
    to_add_.append(len(word) - lenka)
n_tra_lem['resid'] = to_add
n_tra_lem['pref'] = to_pref
n_tra_lem['lenka'] = to_add_

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipyk

In [438]:
print len( v_tra_lem['resid'].value_counts() )
print len( a_tra_lem['resid'].value_counts() )
print len( n_tra_lem['resid'].value_counts() )

55
18
54


In [439]:
len(n_tra_lem)

10267

In [440]:
from sklearn.feature_extraction.text import CountVectorizer

In [441]:
from sklearn.preprocessing import PolynomialFeatures

In [314]:
# pf = PolynomialFeatures(degree=2,interaction_only=True)
# ntrrrr = pd.DataFrame(pf.fit_transform(n_tra_lem.drop([0,1, 'resid'],axis=1).values))

In [442]:
a_tra_lem['target'] = a_tra_lem['resid'] + '+' + map(str,(a_tra_lem['lenka']))

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [443]:
n_tra_lem['target'] = n_tra_lem['resid'] + '+' + map(str,(n_tra_lem['lenka']))

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [300]:
v_tra_lem['target'] = v_tra_lem['resid'] + '+' + map(str,(v_tra_lem['lenka']))

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [359]:
# pf = PolynomialFeatures(degree=2,interaction_only=True)
# ntrrrr = pd.DataFrame(pf.fit_transform(n_tra_lem.drop([0,1, 'resid', 'lenka', 'pref', 'target'],axis=1).values))

In [445]:
test[2] = pos_predicted

In [454]:
test.head()

,Id,X,vowels,consonal,length,doubles,2
0,1,gettonan,3,5,8,1,V
1,2,incidentali,5,6,11,0,A
2,3,involtino,4,5,9,0,V
3,4,lievi,3,2,5,0,N
4,5,comunistizzasse,6,9,15,2,V


In [453]:
sprsed_train_a = scipy.sparse.csr_matrix(a_tra_lem.drop([0,1, 'resid', 'lenka', 'pref', 'target'],axis=1).values)
sprsed_train_n = scipy.sparse.csr_matrix(n_tra_lem.drop([0,1, 'resid', 'lenka', 'pref', 'target'],axis=1).values)
sprsed_train_v = scipy.sparse.csr_matrix(v_tra_lem.drop([0,1, 'resid', 'lenka', 'pref'],axis=1).values)

sprsed_test_a = scipy.sparse.csr_matrix(test[test[2] == 'A'].drop([2, 'Id', 'X'],axis=1).values)
sprsed_test_n = scipy.sparse.csr_matrix(test[test[2] == 'N'].drop([2, 'Id', 'X'],axis=1).values)
sprsed_test_v = scipy.sparse.csr_matrix(test[test[2] == 'V'].drop([2, 'Id', 'X'],axis=1).values)





# ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 5), min_df=1) 
# ngram_vectorizer.fit(a_tra_lem[0]) 

In [449]:
ngram_vectorizer_a = CountVectorizer(analyzer='char_wb', ngram_range=(2, 5), min_df=1) 
ngram_vectorizer_a.fit(a_tra_lem[0])

ngram_vectorizer_n = CountVectorizer(analyzer='char_wb', ngram_range=(2, 5), min_df=1) 
ngram_vectorizer_n.fit(n_tra_lem[0])

ngram_vectorizer_v = CountVectorizer(analyzer='char_wb', ngram_range=(2, 6), min_df=1) 
ngram_vectorizer_v.fit(v_tra_lem[0]) 

CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 6), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [455]:
ngram_train_a = ngram_vectorizer_a.transform(a_tra_lem[0])
ngram_train_n = ngram_vectorizer_n.transform(n_tra_lem[0])
ngram_train_v = ngram_vectorizer_v.transform(v_tra_lem[0])

ngram_test_a = ngram_vectorizer_a.transform(test[test[2] == 'A']['X'])
ngram_test_n = ngram_vectorizer_n.transform(test[test[2] == 'N']['X'])
ngram_test_v = ngram_vectorizer_v.transform(test[test[2] == 'V']['X'])

In [456]:
all_sparsed_tr_a = scipy.sparse.bmat([[ngram_train_a, sprsed_train_a]], format='csr')
all_sparsed_tr_n = scipy.sparse.bmat([[ngram_train_n, sprsed_train_n]], format='csr')
all_sparsed_tr_v = scipy.sparse.bmat([[ngram_train_v, sprsed_train_v]], format='csr')


all_sparsed_te_a = scipy.sparse.bmat([[ngram_test_a, sprsed_test_a]], format='csr')

all_sparsed_te_n = scipy.sparse.bmat([[ngram_test_n, sprsed_test_n]], format='csr')

all_sparsed_te_v = scipy.sparse.bmat([[ngram_test_v, sprsed_test_v]], format='csr')


In [457]:
from sklearn.linear_model import LogisticRegression

labels_for_a&n

In [459]:
%%time
algo_2_a = LogisticRegression(C=.65).fit(all_sparsed_tr_a, a_tra_lem['target'])
label_predicted_a = algo_2_a.predict(all_sparsed_te_a)

CPU times: user 8.84 s, sys: 118 ms, total: 8.96 s
Wall time: 9.1 s


In [460]:
%%time
algo_2_n = LogisticRegression(C=.65).fit(all_sparsed_tr_n, n_tra_lem['target'])
label_predicted_n = algo_2_n.predict(all_sparsed_te_n)

CPU times: user 12.4 s, sys: 63.1 ms, total: 12.5 s
Wall time: 12.6 s


--------------
FOr V, start with predicting len, and than we'll bredict resid

In [464]:
algo_lenka_v = LogisticRegression(C=.65).fit(all_sparsed_tr_v, v_tra_lem['lenka'])
lenka_predicted_v = algo_lenka_v.predict(all_sparsed_te_v)

In [467]:
vvvv_test = test[test[2] == "V"]

In [468]:
vvvv_test['lenka'] = lenka_predicted_v

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [470]:
to_add = []
for word, stemmed in train_lem[[0,1]].values:
    to_add.append(stemmed[get_lcp(word, stemmed):])

train_lem['resid'] = to_add

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [473]:
to_pref = []
for word, lenka in vvvv_test[['X', 'lenka']].values:
    to_pref.append(word[:len(word) - lenka])
vvvv_test['pref'] = to_pref


/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [475]:
vvvv_test.head()

,Id,X,vowels,consonal,length,doubles,2,lenka,pref
0,1,gettonan,3,5,8,1,V,1,gettona
2,3,involtino,4,5,9,0,V,3,involt
4,5,comunistizzasse,6,9,15,2,V,3,comunistizza
5,6,vidimerebbe,5,6,11,1,V,6,vidim
6,7,imbrodan,3,5,8,0,V,1,imbroda


In [477]:
sprsed_vvv =scipy.sparse.csr_matrix(vvvv_test.drop([2, 'Id', 'X', 'pref', 'lenka'],axis=1).values)



In [479]:
ngram_vectorizer_a = CountVectorizer(analyzer='char_wb', ngram_range=(2, 6), min_df=1) 
ngram_vectorizer_a.fit(v_tra_lem['pref'])



CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 6), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [480]:
ngram_train_vvvvvvv = ngram_vectorizer_a.transform(v_tra_lem['pref'])

ngram_test_vvvvvvv = ngram_vectorizer_a.transform(vvvv_test['pref'])


In [481]:
all_sparsed_tr_vvvv = scipy.sparse.bmat([[ngram_train_vvvvvvv, sprsed_train_v]], format='csr')



all_sparsed_te_vvvv = scipy.sparse.bmat([[ngram_test_vvvvvvv, sprsed_vvv]], format='csr')

In [482]:
resid_algo = LogisticRegression(C=.65).fit(all_sparsed_tr_vvvv, v_tra_lem['resid'])
resid_predicted = resid_algo.predict(all_sparsed_te_vvvv)

In [484]:
resid_predicted

array(['re', 'are', 're', ..., 're', 'are', 'are'], dtype=object)

In [562]:
ansa = pd.DataFrame()

In [563]:
ansa['Id'] = test['Id']
ansa['pos'] = pos_predicted

In [564]:
ansa

,Id,pos
0,1,V
1,2,A
2,3,V
3,4,N
4,5,V
5,6,V
6,7,V
7,8,V
8,9,V
9,10,V


In [490]:
label_predicted_a[0]

'e+1'

In [501]:
label_predicted_a[0]

'e+1'

In [568]:
fenal

,0,1,2,X,nf,pos
5282,1,1,re,frastagliavamo,frastagliavamre,V
0,2,1,e,gettonan,gettonae,V
5283,3,3,are,dileggiato,dileggiare,V
3183,4,1,o,debbiavi,debbiavo,V
5284,5,3,re,drogando,drogare,V
5285,6,6,are,sgancerei,sgaare,V
5286,7,1,re,posticiperemmo,posticiperemmre,V
5287,8,0,e,recidivavo,recidivavoe,V
5288,9,3,re,remaron,remare,V
5289,10,4,re,litanie,litre,N


In [567]:
test[test[2] == 'A']

,Id,X,vowels,consonal,length,doubles,2
1,2,incidentali,5,6,11,0,A
20,21,sesquipedale,6,6,12,0,A
21,22,timide,3,3,6,0,A
44,45,plurale,3,4,7,0,A
51,52,secca,2,3,5,1,A
58,59,scomponibili,5,7,12,0,A
68,69,collocabile,5,6,11,1,A
71,72,fornitrici,4,6,10,0,A
84,85,biascicante,5,6,11,0,A
99,100,dittatorio,5,5,10,1,A


In [613]:
del to_merge_all

In [614]:
lenkivatvet = []


for ind , elem in enumerate(test[test[2] == 'A']['Id'].values):
    residka, lena = label_predicted_a[ind].split('+')
    lenkivatvet.append((elem, lena, residka))
    
for ind , elem in enumerate(test[test[2] == 'N']['Id'].values):
    residka, lena = label_predicted_n[ind].split('+')
    lenkivatvet.append((elem, lena, residka))
    
for elem in to_merge_vvv:
    lenkivatvet.append(elem)

In [615]:
to_merge_all = pd.DataFrame(lenkivatvet)

In [616]:
to_merge_all.sort_values(0, inplace=True)

In [620]:
fenal = to_merge_all
fenal['X'] = test['X'].values

In [621]:
fenal.head()

,0,1,2,X
5282,1,1,re,gettonan
0,2,1,e,incidentali
5283,3,3,are,involtino
3183,4,1,o,lievi
5284,5,3,re,comunistizzasse


In [512]:
elem

array(['1', 'e', u'gettonan'], dtype=object)

In [520]:
elem

array(['1', '\xc3\xa0', u'prediligere'], dtype=object)

In [525]:
print elem[2][:-int(elem[0])] + elem[1].decode('utf-8')

prediligerà


In [532]:
elem[2][:0]

u''

In [623]:
fenal_ansa = []
for elem in fenal[[1, 2, 'X']].values:
    if int(elem[0]) == 0:
        fenal_ansa.append(elem[2] + elem[1].decode('utf-8'))
    else:
        fenal_ansa.append(elem[2][:-int(elem[0])] + elem[1].decode('utf-8'))

In [626]:
rili_final = fenal
rili_final['nf'] = fenal_ansa
rili_final['pos'] = ansa['pos'].values

In [628]:
test

,Id,X,vowels,consonal,length,doubles,2
0,1,gettonan,3,5,8,1,V
1,2,incidentali,5,6,11,0,A
2,3,involtino,4,5,9,0,V
3,4,lievi,3,2,5,0,N
4,5,comunistizzasse,6,9,15,2,V
5,6,vidimerebbe,5,6,11,1,V
6,7,imbrodan,3,5,8,0,V
7,8,strillar,2,6,8,1,V
8,9,cifrasti,3,5,8,0,V
9,10,compassavano,5,7,12,1,V


In [632]:
rili_final['label'] = rili_final['nf'] + '+' + rili_final['pos']

In [635]:
rili_final.head()

,0,1,2,X,nf,pos,label
5282,1,1,re,gettonan,gettonare,V,gettonare+V
0,2,1,e,incidentali,incidentale,A,incidentale+A
5283,3,3,are,involtino,involtare,V,involtare+V
3183,4,1,o,lievi,lievo,N,lievo+N
5284,5,3,re,comunistizzasse,comunistizzare,V,comunistizzare+V


In [636]:
to_submit = rili_final[[0,'label']]

In [ ]:
Id,Category

In [638]:
to_submit.columns = ['Id', 'Category']

In [643]:
print u'\xe0'

à


In [646]:
to_submit.head(10)

,Id,Category
5282,1,gettonare+V
0,2,incidentale+A
5283,3,involtare+V
3183,4,lievo+N
5284,5,comunistizzare+V
5285,6,vidimare+V
5286,7,imbrodare+V
5287,8,strillare+V
5288,9,cifrare+V
5289,10,compassare+V


In [650]:
to_submit['Category'].apply(lambda elem: elem.encode('utf-8'))

5282          gettonare+V
0           incidentale+A
5283          involtare+V
3183              lievo+N
5284     comunistizzare+V
5285           vidimare+V
5286          imbrodare+V
5287          strillare+V
5288            cifrare+V
5289         compassare+V
5290             cucire+V
5291           snobbare+V
5292            tessere+V
5293          coagulare+V
5294        somatizzare+V
3184      impoverimento+N
5295           smungere+V
5296          abbuffare+V
5297       meravigliare+V
5298        risucchiare+V
1          sesquipedale+A
2                timido+A
5299           nauseare+V
5300          ingozzare+V
5301            stimare+V
5302        relazionare+V
5303          sorridere+V
5304         illuminare+V
5305          concitare+V
5306          dissecare+V
               ...       
29635     diagnosticare+V
29636     condiscendere+V
5278          banchetto+N
29637          ricocare+V
29638         ammainare+V
29639         azzoppare+V
29640      scoppiettare+V
29641     di

In [652]:
to_submit.to_csv('./my_fst_lemma.csv', index=False, encoding='utf-8')

In [493]:
vvvv_test['preds'] = resid_predicted

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [581]:
to_merge_vvv = vvvv_test[['Id', 'lenka', 'preds']].values

In [582]:
to_merge_vvv

array([[1, 1, 're'],
       [3, 3, 'are'],
       [5, 3, 're'],
       ..., 
       [29659, 1, 're'],
       [29660, 4, 'are'],
       [29661, 7, 'are']], dtype=object)

In [494]:
vvvv_test

,Id,X,vowels,consonal,length,doubles,2,lenka,pref,preds
0,1,gettonan,3,5,8,1,V,1,gettona,re
2,3,involtino,4,5,9,0,V,3,involt,are
4,5,comunistizzasse,6,9,15,2,V,3,comunistizza,re
5,6,vidimerebbe,5,6,11,1,V,6,vidim,are
6,7,imbrodan,3,5,8,0,V,1,imbroda,re
7,8,strillar,2,6,8,1,V,0,strillar,e
8,9,cifrasti,3,5,8,0,V,3,cifra,re
9,10,compassavano,5,7,12,1,V,4,compassa,re
10,11,cucio,3,2,5,0,V,1,cuci,re
11,12,snobbavate,4,6,10,1,V,4,snobba,re


In [487]:
ansa

,Id,pos
0,1,V
1,2,A
2,3,V
3,4,N
4,5,V
5,6,V
6,7,V
7,8,V
8,9,V
9,10,V


In [379]:

# ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=1) 
# ngram_train = ngram_vectorizer.fit_transform(train.surname) 

from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn import linear_model 


X_train, X_test, Y_train, Y_test = train_test_split(all_sparsed, v_tra_lem['lenka'], train_size=.8, random_state=42)#, stratify=v_tra_lem['resid']) 


In [384]:
(len(v_tra_lem) * 0.988981998759 + len(a_tra_lem) * 0.941660701503 + len(n_tra_lem) * 0.93135345667 )/ len(train)

0.9786198659989308

In [381]:
%%time
lr = LogisticRegression(C=.65).fit(X_train, Y_train) 
print (accuracy_score(Y_test, lr.predict(X_test)))

0.947599834471
CPU times: user 49.8 s, sys: 153 ms, total: 49.9 s
Wall time: 50.1 s


In [374]:
preds_0 = lr.predict(X_test)

In [375]:

# ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=1) 
# ngram_train = ngram_vectorizer.fit_transform(train.surname) 

from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn import linear_model 


X_train, X_test, Y_train, Y_test = train_test_split(all_sparsed, v_tra_lem['resid'], train_size=.8, random_state=42)#, stratify=v_tra_lem['resid']) 



In [383]:
(1 - 0.996172149803) * X_test.shape[0]

74.00000000840494

In [247]:
to_add = []
for word, stemmed in train_lem[[0,1]].values:
    to_add.append(stemmed[get_lcp(word, stemmed):])

train_lem['resid'] = to_add

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [208]:
sprsed_train_v = scipy.sparse.csr_matrix(train_lem.drop([0,1,2, 'resid'],axis=1).values)
# sprsed_train_v = scipy.sparse.csr_matrix(ntrrrr.values)



ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 4), min_df=1) 
ngram_vectorizer.fit(train_lem[0]) 

CountVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 4), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [209]:
ngram_train = ngram_vectorizer.transform(train_lem[0])



all_sparsed = scipy.sparse.bmat([[ngram_train, sprsed_train_v]], format='csr')
all_sparsed

<120897x23132 sparse matrix of type '<type 'numpy.int64'>'
	with 4164263 stored elements in Compressed Sparse Row format>

In [382]:
test

,Id,X
0,1,gettonan
1,2,incidentali
2,3,involtino
3,4,lievi
4,5,comunistizzasse
5,6,vidimerebbe
6,7,imbrodan
7,8,strillar
8,9,cifrasti
9,10,compassavano
